In [11]:
import json
import datetime
import requests
import pandas as pd
import numpy as np

## Collect Data

In [2]:
# Read in data from BitStamp
HIST = json.loads(open('/Users/cn/git/crypto-etl/bitstamp_daily.json').read())

# Collect data from coinbin
RCNT = json.loads(requests.get('https://coinbin.org/btc/history').text)

### Create DataFrames

In [34]:
# Create some DataFrames
df_hist = pd.DataFrame.from_dict(HIST)
df_rcnt = pd.DataFrame.from_dict(RCNT['history'])

In [35]:
# Convert to timestamps
df_hist.dt = pd.to_datetime(df_hist.dt, format='%Y-%m-%d')
df_rcnt.timestamp = pd.to_datetime(df_rcnt.timestamp, infer_datetime_format=True)

In [36]:
# Get rid of extra cols
df_rcnt = df_rcnt[['timestamp', 'value']]
df_rcnt.rename(columns={'timestamp': 'dt', 'value': 'price'}, inplace=True)

In [38]:
df_hist.head()

,dt,price
0,2011-09-13,5.874167
1,2011-09-14,5.582143
2,2011-09-15,5.120000
3,2011-09-16,4.835000
4,2011-09-17,4.870000


### Compare dates and combine

In [39]:
# See how far recent data goes back
min_rcnt = min(df_rcnt.dt)

Timestamp('2014-02-17 23:54:09.761687')

In [41]:
# Filter the historical data based on min_recent
df_hist = df_hist[df_hist['dt'] < min_rcnt]

In [44]:
# Filter the last day
max_hist = max(df_hist.dt)
df_hist = df_hist[df_hist['dt'] < max_hist]

In [46]:
# Concatenate the DataFrames
df_all = pd.concat([df_rcnt, df_hist])

## Clean and publish

In [52]:
df_all['dt_str'] = df_all['dt'].dt.strftime(date_format='%Y-%m-%d')

In [53]:
df_all.head()

,dt,price,dt_str
0,2017-09-16 00:00:50.103574,3637.52,2017-09-16
1,2017-09-15 00:00:50.159154,3154.95,2017-09-15
2,2017-09-14 00:01:10.028555,3882.59,2017-09-14
3,2017-09-13 00:00:40.119372,4130.81,2017-09-13
4,2017-09-12 00:00:53.248195,4163.20,2017-09-12


In [55]:
# Publish the data
records = df_all[['dt_str', 'price']].to_dict(orient='records')
with open('/Users/cn/git/crypto-etl/btcData.json', 'w') as f:
    f.write(json.dumps(records))